In [43]:
!pip install optuna

In [44]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [45]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [46]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [47]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [48]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2025-07-07 06:26:26,507] A new study created in memory with name: no-name-2661f719-2e3a-4504-acc5-75e5353ba3df
[I 2025-07-07 06:26:27,174] Trial 0 finished with value: 0.7560521415270017 and parameters: {'n_estimators': 57, 'max_depth': 4}. Best is trial 0 with value: 0.7560521415270017.
[I 2025-07-07 06:26:28,263] Trial 1 finished with value: 0.7597765363128491 and parameters: {'n_estimators': 150, 'max_depth': 4}. Best is trial 1 with value: 0.7597765363128491.
[I 2025-07-07 06:26:29,113] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 81, 'max_depth': 18}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-07-07 06:26:31,246] Trial 3 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 158, 'max_depth': 14}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-07-07 06:26:33,990] Trial 4 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 175, 'max_depth': 11}. Best is trial 2 with value: 0.77281191

In [49]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7839851024208566
Best hyperparameters: {'n_estimators': 119, 'max_depth': 15}


In [50]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


## Samplers in Optuna

In [51]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [52]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-07-07 06:27:17,793] A new study created in memory with name: no-name-05d26124-e0ae-4607-9ebf-9bb78ffe0d74
[I 2025-07-07 06:27:19,141] Trial 0 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 174, 'max_depth': 7}. Best is trial 0 with value: 0.7728119180633147.
[I 2025-07-07 06:27:20,699] Trial 1 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 141, 'max_depth': 16}. Best is trial 0 with value: 0.7728119180633147.
[I 2025-07-07 06:27:21,914] Trial 2 finished with value: 0.7541899441340782 and parameters: {'n_estimators': 99, 'max_depth': 3}. Best is trial 0 with value: 0.7728119180633147.
[I 2025-07-07 06:27:22,712] Trial 3 finished with value: 0.7560521415270017 and parameters: {'n_estimators': 65, 'max_depth': 3}. Best is trial 0 with value: 0.7728119180633147.
[I 2025-07-07 06:27:24,409] Trial 4 finished with value: 0.7802607076350093 and parameters: {'n_estimators': 130, 'max_depth': 7}. Best is trial 4 with value: 0.7802607076

In [53]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 130, 'max_depth': 7}


In [54]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


In [55]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [56]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2025-07-07 06:28:09,572] A new study created in memory with name: no-name-6f4875b1-9d1b-4e7d-b79e-d210be739db2
[I 2025-07-07 06:28:10,146] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-07-07 06:28:11,093] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-07-07 06:28:11,431] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-07-07 06:28:12,109] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-07-07 06:28:12,774] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [57]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [58]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


## Optuna Visualizations

In [59]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [60]:
# 1. Optimization History
plot_optimization_history(study).show()

In [61]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [62]:
# 3. Slice Plot
plot_slice(study).show()

In [63]:
# 4. Contour Plot
plot_contour(study).show()

In [64]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

## Optimizing Multiple ML Models

In [65]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [66]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [67]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-07-07 06:28:23,136] A new study created in memory with name: no-name-a92e8b19-635e-4632-a300-9b0f58e5ae67
[I 2025-07-07 06:28:23,181] Trial 0 finished with value: 0.7094972067039106 and parameters: {'classifier': 'SVM', 'C': 10.405414166461194, 'kernel': 'poly', 'gamma': 'scale'}. Best is trial 0 with value: 0.7094972067039106.
[I 2025-07-07 06:28:26,199] Trial 1 finished with value: 0.7281191806331471 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 183, 'learning_rate': 0.07120154187349789, 'max_depth': 19, 'min_samples_split': 6, 'min_samples_leaf': 6}. Best is trial 1 with value: 0.7281191806331471.
[I 2025-07-07 06:28:27,125] Trial 2 finished with value: 0.7653631284916201 and parameters: {'classifier': 'RandomForest', 'n_estimators': 184, 'max_depth': 16, 'min_samples_split': 2, 'min_samples_leaf': 5, 'bootstrap': False}. Best is trial 2 with value: 0.7653631284916201.
[I 2025-07-07 06:28:27,216] Trial 3 finished with value: 0.7858472998137801 and parame

In [68]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.14737019274538027, 'kernel': 'linear', 'gamma': 'auto'}
Best trial accuracy: 0.7895716945996275


In [69]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.709497,2025-07-07 06:28:23.140384,2025-07-07 06:28:23.181415,0 days 00:00:00.041031,10.405414,NaN,SVM,scale,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
1,1,0.728119,2025-07-07 06:28:23.182337,2025-07-07 06:28:26.199383,0 days 00:00:03.017046,NaN,NaN,GradientBoosting,NaN,NaN,0.071202,19.0,6.0,6.0,183.0,COMPLETE
2,2,0.765363,2025-07-07 06:28:26.200365,2025-07-07 06:28:27.125396,0 days 00:00:00.925031,NaN,False,RandomForest,NaN,NaN,NaN,16.0,5.0,2.0,184.0,COMPLETE
3,3,0.785847,2025-07-07 06:28:27.126434,2025-07-07 06:28:27.216178,0 days 00:00:00.089744,16.254868,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
4,4,0.754190,2025-07-07 06:28:27.217345,2025-07-07 06:28:30.413375,0 days 00:00:03.196030,NaN,NaN,GradientBoosting,NaN,NaN,0.010628,16.0,5.0,4.0,198.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.789572,2025-07-07 06:29:06.989097,2025-07-07 06:29:07.023264,0 days 00:00:00.034167,0.133540,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.711359,2025-07-07 06:29:07.024176,2025-07-07 06:29:07.067019,0 days 00:00:00.042843,0.101564,NaN,SVM,auto,rbf,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.741155,2025-07-07 06:29:07.067973,2025-07-07 06:29:09.248184,0 days 00:00:02.180211,NaN,NaN,GradientBoosting,NaN,NaN,0.119987,13.0,7.0,7.0,155.0,COMPLETE
98,98,0.785847,2025-07-07 06:29:09.249148,2025-07-07 06:29:09.287203,0 days 00:00:00.038055,0.231257,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [70]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,78
GradientBoosting,12
RandomForest,10


In [71]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.744879
RandomForest,0.763128
SVM,0.770424


In [72]:
# 1. Optimization History
plot_optimization_history(study).show()

In [73]:
# 3. Slice Plot
plot_slice(study).show()

In [74]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [75]:
! pip install optuna-integration[xgboost]

In [76]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")


[I 2025-07-07 06:29:18,334] A new study created in memory with name: no-name-5bf187d0-b65b-41cc-b3db-e785d24ac2f1


[0]	train-mlogloss:1.05087	eval-mlogloss:1.04798
[1]	train-mlogloss:1.01523	eval-mlogloss:1.00837
[2]	train-mlogloss:0.97239	eval-mlogloss:0.96397
[3]	train-mlogloss:0.92902	eval-mlogloss:0.91877
[4]	train-mlogloss:0.89188	eval-mlogloss:0.88018
[5]	train-mlogloss:0.85442	eval-mlogloss:0.84093
[6]	train-mlogloss:0.82032	eval-mlogloss:0.80564
[7]	train-mlogloss:0.78928	eval-mlogloss:0.77362
[8]	train-mlogloss:0.75956	eval-mlogloss:0.74184
[9]	train-mlogloss:0.73215	eval-mlogloss:0.71292
[10]	train-mlogloss:0.70586	eval-mlogloss:0.68394
[11]	train-mlogloss:0.67983	eval-mlogloss:0.65578
[12]	train-mlogloss:0.65730	eval-mlogloss:0.63181
[13]	train-mlogloss:0.63412	eval-mlogloss:0.60709
[14]	train-mlogloss:0.61106	eval-mlogloss:0.58262
[15]	train-mlogloss:0.58981	eval-mlogloss:0.56028
[16]	train-mlogloss:0.56973	eval-mlogloss:0.53931
[17]	train-mlogloss:0.55125	eval-mlogloss:0.51899
[18]	train-mlogloss:0.53271	eval-mlogloss:0.49981
[19]	train-mlogloss:0.51738	eval-mlogloss:0.48413
[20]	train

[I 2025-07-07 06:29:19,845] Trial 0 finished with value: 1.0 and parameters: {'lambda': 0.3919100241601361, 'alpha': 3.247354749129297e-06, 'eta': 0.0393041866521694, 'gamma': 2.4164675396033107e-08, 'max_depth': 4, 'min_child_weight': 7, 'subsample': 0.5496288307797373, 'colsample_bytree': 0.8361034931158132}. Best is trial 0 with value: 1.0.


[0]	train-mlogloss:1.07563	eval-mlogloss:1.07377
[1]	train-mlogloss:1.05364	eval-mlogloss:1.05057
[2]	train-mlogloss:1.03192	eval-mlogloss:1.02802
[3]	train-mlogloss:1.01016	eval-mlogloss:1.00580
[4]	train-mlogloss:0.98996	eval-mlogloss:0.98482
[5]	train-mlogloss:0.96953	eval-mlogloss:0.96273
[6]	train-mlogloss:0.95037	eval-mlogloss:0.94252
[7]	train-mlogloss:0.93226	eval-mlogloss:0.92338
[8]	train-mlogloss:0.91458	eval-mlogloss:0.90429
[9]	train-mlogloss:0.89686	eval-mlogloss:0.88517
[10]	train-mlogloss:0.87985	eval-mlogloss:0.86776
[11]	train-mlogloss:0.86335	eval-mlogloss:0.85041
[12]	train-mlogloss:0.84876	eval-mlogloss:0.83590
[13]	train-mlogloss:0.83288	eval-mlogloss:0.81914
[14]	train-mlogloss:0.81750	eval-mlogloss:0.80307
[15]	train-mlogloss:0.80267	eval-mlogloss:0.78757
[16]	train-mlogloss:0.78783	eval-mlogloss:0.77180
[17]	train-mlogloss:0.77386	eval-mlogloss:0.75680
[18]	train-mlogloss:0.76315	eval-mlogloss:0.74592
[19]	train-mlogloss:0.74968	eval-mlogloss:0.73178
[20]	train

[I 2025-07-07 06:29:22,936] Trial 1 finished with value: 1.0 and parameters: {'lambda': 6.763293494361026e-07, 'alpha': 0.5928166978573554, 'eta': 0.018601493058450887, 'gamma': 1.9546240490581244e-05, 'max_depth': 4, 'min_child_weight': 9, 'subsample': 0.6560618306613367, 'colsample_bytree': 0.7960885233294196}. Best is trial 0 with value: 1.0.


...

[0]	train-mlogloss:0.96710	eval-mlogloss:0.95582
[1]	train-mlogloss:0.85914	eval-mlogloss:0.83866


[I 2025-07-07 06:29:49,599] Trial 48 pruned. Trial was pruned at iteration 2.


[0]	train-mlogloss:1.06277	eval-mlogloss:1.06662
[1]	train-mlogloss:1.01643	eval-mlogloss:1.01585


[I 2025-07-07 06:29:49,740] Trial 49 pruned. Trial was pruned at iteration 2.


Best trial: {'lambda': 0.3919100241601361, 'alpha': 3.247354749129297e-06, 'eta': 0.0393041866521694, 'gamma': 2.4164675396033107e-08, 'max_depth': 4, 'min_child_weight': 7, 'subsample': 0.5496288307797373, 'colsample_bytree': 0.8361034931158132}
Best accuracy: 1.0


In [77]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()